#1 Introducción

En el siguiente cuaderno se realiza una suma de matrices en python implementando paralelismo utilizando la CPU.

#2 Armado del ambiente

No son necesarios, ejecuciones previas del armado del ambiente.

#3 Desarrollo

In [17]:
#@title Parametros: { vertical-output: true, display-mode: "both" }
ordenMatriz =  #@param {type:"number"}

from datetime import datetime
import threading
import numpy
import queue

tiempoTotal = datetime.now()

tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
try:
  cola1 = queue.Queue()
  cola2 = queue.Queue()

  def operacionesMatriz1(matriz,nombre,ordenMatriz):
    print("hola soy:",nombre)
    print(matriz)
    for i in range(0,ordenMatriz):
      for j in range(0,ordenMatriz):
        cola1.put(matriz[i][j])

  def operacionesMatriz2(matriz,nombre,ordenMatriz):
    print("hola soy:",nombre)
    print(matriz)
    for i in range(0,ordenMatriz):
      for j in range(0,ordenMatriz):
        cola2.put(matriz[i][j])

  matriz1 = numpy.random.random((ordenMatriz,ordenMatriz))
  matriz1 = matriz1.astype(numpy.float32())
  matriz2 = numpy.random.random((ordenMatriz,ordenMatriz))
  matriz2 = matriz2.astype(numpy.float32())
  matrizR = numpy.empty_like(matriz1)

  thread1 = threading.Thread(target=operacionesMatriz1,args=(matriz1,"hilo1",ordenMatriz,))
  thread1.start()

  thread2 = threading.Thread(target=operacionesMatriz2,args=(matriz2,"hilo2",ordenMatriz,))
  thread2.start()
  thread2.join()

  print("")
  print("Soy el hilo principal y voy a hacer el calculo de la suma de matrices")
  print("")
  for i in range(0,ordenMatriz):
    for j in range(0,ordenMatriz):
        numeroM1 = cola1.get()
        numeroM2 = cola2.get()
        matrizR[i][j] = (numeroM1+numeroM2)

  tiempoTotal = datetime.now() - tiempoTotal

  print("Matriz Resultado de la suma: ")
  print(matrizR)
  print("")
  print("Tiempo Total: ",tiempo_en_ms(tiempoTotal),"[ms]")

except ValueError:
  print("se ha ingresado un numero de orden menor a 0")
  print("ingrese un numero de orden de la matriz igual o mayor a 0")


hola soy:hola soy: hilo2
 hilo1[[0.43295136 0.05331363 0.1634922  ... 0.88171923 0.8889116  0.11814816]
 [0.6318692  0.44612226 0.37465385 ... 0.02472186 0.39573568 0.15067437]
 [0.83002746 0.4696359  0.7987387  ... 0.76341695 0.28646576 0.17723168]
 ...
 [0.32404238 0.7472796  0.5661467  ... 0.29974464 0.7296248  0.22886483]
 [0.0924739  0.153247   0.66189957 ... 0.07616069 0.7137064  0.94027954]
 [0.44962704 0.831618   0.5982383  ... 0.80603546 0.10723941 0.8530909 ]]

[[0.7968799  0.69426376 0.3581042  ... 0.14713463 0.56093574 0.58747715]
 [0.23858441 0.9860165  0.56435734 ... 0.43881136 0.5712752  0.99246013]
 [0.9556674  0.24197043 0.42905012 ... 0.03798871 0.3638672  0.8544714 ]
 ...
 [0.4371943  0.97784305 0.47951266 ... 0.28249145 0.2960809  0.79647887]
 [0.9429526  0.28920904 0.401014   ... 0.97755927 0.5723605  0.13437681]
 [0.15926395 0.05826841 0.61664426 ... 0.9377772  0.17433815 0.8139419 ]]

Soy el hilo principal y voy a hacer el calculo de la suma de matrices

Matriz R


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño del vector desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( ordenMatriz ) | Inicializa las matrices
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel donde se transpone la matriz
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_hilo/dim_bloque   | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU      |  print()               | Informo los resultados.

#5 Conclusiones

#6 Bibliografia